# TFA: Time-Frequency Analysis

An introduction to the TFA toolbox. This gives a basic demonstration of a wavelet analysis applied to the MAGx_LR data (magnetic field 1Hz). For more details, see the following pages/notebooks.

In [ ]:
import datetime as dt

from swarmpal.io import create_paldata, PalDataItem
from swarmpal.toolboxes import tfa

## Fetching data

`create_paldata` and `PalDataItem.from_vires` are used together to pull the desired data from VirES.

- `collection="SW_OPER_MAGA_LR_1B"` selects the MAGx_LR dataset from Swarm Alpha
- `measurements=["B_NEC"]` selects the magnetic vector, B_NEC, which we will analyse
- We provide a model choice with `models=["Model='CHAOS-Core'+'CHAOS-Static'"]`. This can be changed to e.g. `models=["Model=CHAOS"]` which provides the full CHAOS model, including the magnetosphere. The string provided here is a VirES-compatible model string which specifies the chosen model (see [viresclient](https://viresclient.readthedocs.io/en/latest/available_parameters.html#models) for more information).
- The start and end times select the period we want to analyse
- The `pad_times` option supplies an extra padding of 3 hours of data before and after the analysis window, which prevents edge effects in the wavelet analysis

Running this code creates a `DataTree` object in the `data` variable. This contains all the data we will use in our analysis.

In [ ]:
data = create_paldata(
    PalDataItem.from_vires(
        collection="SW_OPER_MAGA_LR_1B",
        measurements=["B_NEC"],
        models=["Model='CHAOS-Core'+'CHAOS-Static'"],
        auxiliaries=["QDLat", "MLT"],
        start_time=dt.datetime(2015, 3, 14),
        end_time=dt.datetime(2015, 3, 14, 3, 59, 59),
        pad_times=(dt.timedelta(hours=3), dt.timedelta(hours=3)),
        server_url="https://vires.services/ows",
        options=dict(asynchronous=False, show_progress=False),
    )
)
print(data)

## Applying processes

Now we will apply some processes to our data. These are all available within the `tfa` submodule which we imported earlier.

### TFA: Preprocess

First we *preprocess* the data. To do this we use `Preprocess`, and, using `.set_config`, supply a configuration to control how that should behave. This will create a function that we can apply to the data - we will call this function `p1` since it is the first process we will apply.

(Strictly speaking `p1` is a special object rather than a regular function in Python, but has been made *callable* so that we can use it like `p1(data)` to apply it to `data`)

Some of the options are used here, but refer to the other demonstrations for more detail.

In [ ]:
p1 = tfa.processes.Preprocess()
p1.set_config(
    # Identifies the data we are working with
    # (this dataset must be available in the DataTree above)
    dataset="SW_OPER_MAGA_LR_1B",
    # Selects the variable we want to process
    # B_NEC_res_Model is a special variable that is added to the data
    #   when we also specify remove_model=True
    active_variable="B_NEC_res_Model",
    # Selects which component (0, 1, 2) to process
    # With the NEC vectors, 2 will refer to the "C" (centre / downward) component
    active_component=2,
    # Identifies the sampling rate (in Hz) of the data
    sampling_rate=1,
    # Removes the magnetic model from the data
    # i.e. gets the data-model residual / perturbation
    remove_model=True,
)

Now we have a process `p1`, we can apply that to the data:

In [ ]:
p1(data);

The preprocessing plays a number of roles but most importantly it sets a new array, `TFA_Variable`, in the data, which will be used in the following steps.

This data can be inspected directly using tools available through xarray:

In [ ]:
data["SW_OPER_MAGA_LR_1B"]["TFA_Variable"]

In [ ]:
data["SW_OPER_MAGA_LR_1B"]["TFA_Variable"].plot.line(x="TFA_Time");

The TFA toolbox also provides a function to inspect the data, `tfa.plotting.time_series`:

In [ ]:
tfa.plotting.time_series(data);

We can see an unphysical spike in the data. In this case we could have removed this bad data in the data request step (by filtering according to flag values), but here we will demonstrate the functionality of the TFA toolbox in removing this spike.

### TFA: Clean

Just as with `Preprocess`, there is `Clean` which provides a data cleaning function to remove outliers.

In [ ]:
p2 = tfa.processes.Clean()
p2.set_config(
    window_size=300,
    method="iqr",
    multiplier=1,
)
p2(data)
tfa.plotting.time_series(data);

### TFA: Filter

Next we use `Filter`, which applies a high-pass filter to the data.

In [ ]:
p3 = tfa.processes.Filter()
p3.set_config(
    cutoff_frequency=20 / 1000,
)
p3(data)
tfa.plotting.time_series(data);

### TFA: Wavelet

Now we get to the most interesting part: applying a wavelet analysis.

In [ ]:
p4 = tfa.processes.Wavelet()
p4.set_config(
    min_frequency=20 / 1000,
    max_frequency=100 / 1000,
    dj=0.1,
)
p4(data);

The results of the wavelet analysis are stored as a spectrum in a new `wavelet_power` array:

In [ ]:
data["SW_OPER_MAGA_LR_1B"]["wavelet_power"]

And we can view it with `tfa.plotting.spectrum`:

In [ ]:
tfa.plotting.spectrum(data)

## Plotting

We showed basic usage of `tfa.plotting.time_series` and `tfa.plotting.spectrum` above. These have some extra options to control their behaviour. For example:

- Adding extra x axes with `extra_x`
- Selecting a specific time window with `tlims`

In [ ]:
tfa.plotting.time_series(
    data,
    extra_x=("QDLat", "MLT", "Latitude"),
    tlims=("2015-03-14T03:00:00", "2015-03-14T03:30:00"),
);

There is also a quicklook plot that combines the time series, spectrum, and wave power (from `tfa.plotting.wave_index`) into one figure:

In [ ]:
tfa.plotting.quicklook(
    data,
    tlims=("2015-03-14T03:00:00", "2015-03-14T03:30:00"),
    extra_x=("QDLat", "MLT", "Latitude"),
);